<a href="https://colab.research.google.com/github/dt-cs/CMPSC-497/blob/main/S25_CMPSC_497_HW01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task description:
Train logistic regression or SVM classifier on word embeddings for POS tagging task, get reasonable performance (e.g., ~70%)

In [1]:
import numpy as np
import pandas as pd
import gensim.downloader as api
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whic

In [3]:
from datasets import load_dataset

In [6]:
# Downloading Word2Vec model
word2vec_model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [7]:
# Load dataset
ds = load_dataset("batterydata/pos_tagging")

# Use only the first 1000 rows
train_data = ds["train"].select(range(1000))
test_data = ds["test"]

def word_to_embedding(word):
    # Use Word2Vec to get the vector for the word
    try:
        return word2vec_model[word]  # Returns a 300-dimensional embedding vector
    except KeyError:
        return np.zeros(300)  # Return zero vector if the word is not in the model's vocabulary

In [10]:
# Prepare training data
X_train, y_train = [], []
for item in train_data:
    words = item["words"]
    labels = item["labels"]
    for word, label in zip(words, labels):
        X_train.append(word_to_embedding(word))
        y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [11]:
# Prepare test data
X_test, y_test = [], []
for item in test_data:
    words = item["words"]
    labels = item["labels"]
    for word, label in zip(words, labels):
        X_test.append(word_to_embedding(word))
        y_test.append(label)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [16]:
# Train a logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [17]:
# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 67.94%


In [18]:
# Function to predict POS tag
def predict_pos(word):
    embedding = word_to_embedding(word).reshape(1, -1)
    return model.predict(embedding)[0]

In [19]:
# Example usage
word_to_test = "expected"
print(f"Predicted POS for '{word_to_test}': {predict_pos(word_to_test)}")

Predicted POS for 'expected': VBN
